In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
import numpy as np
import re 
import string
from string import punctuation
from collections import Counter

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip '/content/drive/My Drive/jigsaw-toxic-comment-train.csv.zip' 
!unzip '/content/drive/My Drive/jigsaw-unintended-bias-train.csv.zip' 
!unzip '/content/drive/My Drive/test.csv.zip' 
!unzip '/content/drive/My Drive/validation.csv.zip'

Archive:  /content/drive/My Drive/jigsaw-toxic-comment-train.csv.zip
  inflating: jigsaw-toxic-comment-train.csv  
Archive:  /content/drive/My Drive/jigsaw-unintended-bias-train.csv.zip
  inflating: jigsaw-unintended-bias-train.csv  
Archive:  /content/drive/My Drive/test.csv.zip
  inflating: test.csv                
Archive:  /content/drive/My Drive/validation.csv.zip
  inflating: validation.csv          


In [ ]:
comment_train = pd.read_csv('jigsaw-toxic-comment-train.csv')

bias_train = pd.read_csv('jigsaw-unintended-bias-train.csv')

test_f = pd.read_csv('test.csv')

val = pd.read_csv('validation.csv')

###Анализ всех дополнительных колонок тональности (как проявляется тот или иной тип токсичности, как в данных это представлено, какие есть пограничные случаи) - 2.5 балла


In [ ]:
def normalize(text):
  text = re.sub('\n|\t|\s\"', ' ', text)
  text = re.sub(f'[{punctuation}]', ' ', text)
  text = re.sub('\d+', ' ', text)
  text = text.lower()
  text = re.sub('\s+', ' ', text)

  return text

In [ ]:
comment_train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [ ]:
def oth_cols_an(df):
  tox = df[df['toxic'] == 1]
  LTox = len(tox)
  non_tox = df[df['toxic'] == 0]
  LNon_tox = len(non_tox)
  print('TOXIC/NON-TOXIC ratio: ', LTox/LNon_tox)
  severeTox = df[df['severe_toxic'] == 1]
  tox_with_severeTox = severeTox[severeTox['toxic'] == 1]
  print('Toxic comments with severe_toxic ratio: ', len(tox_with_severeTox)/len(severeTox))
  obscene = df[df['obscene'] == 1]
  #print(len(obscene))
  Toxscene = obscene[obscene['toxic'] == 1]
  print('Toxic comments with obscene ratio:', len(Toxscene)/len(obscene))
  threat = df[df['threat'] == 1]
  #print(len(threat))
  Tox_threat = threat[threat['toxic'] == 1]
  print('Toxic comments with threat ratio:', len(Tox_threat)/len(threat))
  insult = df[df['insult'] == 1]
  #print(len(insult))
  Tox_insult = insult[insult['toxic'] == 1]
  print('Toxic comments with insult ratio:', len(Tox_insult)/len(insult))
  identity_hate = df[df['identity_hate'] == 1]
  #print(len(identity_hate))
  Tox_identity_hate = identity_hate[identity_hate['toxic'] == 1]
  print('Toxic comments with identity hate ratio:', len(Tox_identity_hate)/len(identity_hate))

In [ ]:
oth_cols_an(comment_train)

TOXIC/NON-TOXIC ratio:  0.10577498577894294
Toxic comments with severe_toxic ratio:  1.0
Toxic comments with obscene ratio: 0.9515650741350906
Toxic comments with threat ratio: 0.9492017416545718
Toxic comments with insult ratio: 0.9453290870488322
Toxic comments with identity hate ratio: 0.942371280113368


Как и следовало ожидать, если коммент severe_toxic, то он при этом и просто toxic. С другими же колонками всё не так однозначно. Обсценная лексика действительно не всегда указывает на токсичность, но вот угрозы и identity hate (особенно последнее) по идее должны обозначать токсичность. 

Посмотрим на самые частотные слова в комментах с тегом sever_toxic, так как появление его гарантирует, что комментарий будет токсичным

In [ ]:
import nltk
from nltk.corpus import stopwords
en_stops2 = stopwords.words('english')

In [ ]:
def common_tok(df):
  words = []
  for text in df['comment_text']:
    text = normalize(text)
    text = text.split()
    for word in text:
      if word not in en_stops and word not in en_stops2:
        words.append(word)
      else:
        continue

  print(Counter(words).most_common(1000))
  return words

In [ ]:
severeTox = comment_train[comment_train['severe_toxic'] == 1]
tox_with_severeTox = severeTox[severeTox['toxic'] == 1]

severe = common_tok(tox_with_severeTox)

[('fuck', 8953), ('suck', 3010), ('ass', 2627), ('shit', 2441), ('die', 2167), ('fucking', 2131), ('bitch', 2075), ('nigger', 1992), ('u', 1884), ('faggot', 1850), ('go', 1488), ('cunt', 1146), ('sucks', 1010), ('wikipedia', 901), ('fucker', 840), ('cock', 816), ('kill', 804), ('gay', 764), ('dick', 722), ('like', 686), ('fucksex', 624), ('bastard', 622), ('yourselfgo', 621), ('must', 577), ('asshole', 557), ('penis', 545), ('fat', 540), ('mother', 538), ('eat', 529), ('piece', 524), ('cocksucker', 519), ('mothjer', 489), ('stupid', 484), ('get', 443), ('bitches', 441), ('huge', 429), ('dog', 425), ('niggerjew', 424), ('fuk', 419), ('shut', 417), ('vandal', 417), ('pussy', 416), ('damn', 413), ('traitor', 407), ('niggers', 404), ('rape', 397), ('anal', 390), ('going', 377), ('offfuck', 360), ('mexicans', 359), ('little', 354), ('pro', 350), ('hanibal', 348), ('assad', 347), ('niggas', 344), ('dickhead', 344), ('idiot', 332), ('whore', 332), ('block', 327), ('motherfucker', 318), ('bush

Вот тут неплохой список токсичных слов. Значит, если среди важных фич (далее, через eli5) попадается что-то из этого списка, их в ни коем случае не стоит подчищать. Однако если и менее однозначные токены, встречающиеся в пограничных случаях. 

Теперь рассмотрим случаи, когда единичка во вспомогательных колонках (obscene, threat, insult, identity_hate) не делает комментарий токсичным. 

In [ ]:
def non_tox_an(df):
  obscene = df[df['obscene'] == 1]
  #print(len(obscene))
  Non_tox_obscene = obscene[obscene['toxic'] == 0]
  print('Non-toxic comments with obscene ratio:', len(Non_tox_obscene)/len(obscene))
  threat = df[df['threat'] == 1]
  #print(len(threat))
  Non_tox_threat = threat[threat['toxic'] == 0]
  print('Non-toxic comments with threat ratio:', len(Non_tox_threat)/len(threat))
  insult = df[df['insult'] == 1]
  #print(len(insult))
  Non_tox_insult = insult[insult['toxic'] == 0]
  print('Non_toxic comments with insult ratio:', len(Non_tox_insult)/len(insult))
  identity_hate = df[df['identity_hate'] == 1]
  #print(len(identity_hate))
  Non_tox_identity_hate = identity_hate[identity_hate['toxic'] == 0]
  print('Non-toxic comments with identity hate ratio:', len(Non_tox_identity_hate)/len(identity_hate))

  return Non_tox_obscene, Non_tox_threat, Non_tox_insult, Non_tox_identity_hate 



In [ ]:
Non_tox_obscene, Non_tox_threat, Non_tox_insult, Non_tox_identity_hate = non_tox_an(comment_train)

Non-toxic comments with obscene ratio: 0.04843492586490939
Non-toxic comments with threat ratio: 0.05079825834542816
Non_toxic comments with insult ratio: 0.05467091295116773
Non-toxic comments with identity hate ratio: 0.05762871988663203


In [ ]:
NT_obs = common_tok(Non_tox_obscene)
NT_thr = common_tok(Non_tox_threat)
NT_ins = common_tok(Non_tox_insult)
NT_ih = common_tok(Non_tox_identity_hate)

[('twat', 475), ('like', 180), ('trevor', 171), ('thorpe', 171), ('page', 163), ('wikipedia', 158), ('article', 151), ('user', 137), ('one', 116), ('talk', 114), ('people', 112), ('think', 108), ('know', 104), ('get', 101), ('please', 96), ('dicks', 90), ('see', 89), ('would', 88), ('edit', 88), ('bbb', 88), ('neiln', 88), ('licks', 85), ('fuck', 81), ('time', 79), ('make', 78), ('shit', 78), ('even', 77), ('ass', 68), ('go', 66), ('way', 64), ('personal', 64), ('really', 62), ('also', 62), ('well', 61), ('bitch', 60), ('stop', 57), ('fucking', 56), ('right', 55), ('say', 54), ('something', 53), ('thing', 53), ('use', 53), ('want', 53), ('yeah', 53), ('information', 52), ('read', 51), ('much', 50), ('good', 50), ('back', 50), ('going', 48), ('never', 47), ('da', 46), ('said', 45), ('made', 45), ('u', 44), ('first', 44), ('wrong', 43), ('blocked', 43), ('someone', 43), ('another', 43), ('life', 43), ('look', 43), ('lucilio', 43), ('filho', 43), ('puta', 43), ('got', 42), ('mean', 41), (

Вот эти слова уже мало что обозначают. То есть часть из них (ту, что не пересекается с множеством severe) можно убрать. 

###Бейзлайн модель из sklearn (векторайзер + модель) с отбором признаков (через l1 регуляризацию, на глаз через анализ важных параметров или через permutation importance) - 2 балл

In [ ]:
sample1 = comment_train.sample(frac=0.04, replace=True, random_state=1)

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(max_features=15000, ngram_range=(1, 3))

In [ ]:
X = count_vect.fit_transform(sample1['comment_text'])
y = sample1['toxic']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)

In [ ]:
stratified_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

Логрег

In [2]:
from sklearn.metrics import f1_score
from sklearn import linear_model

In [ ]:
lr = linear_model.LogisticRegression() 
lr.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lr, X_train, y_train, cv=stratified_folds, scoring='roc_auc')
print("ROC: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
scores = cross_val_score(lr, X_train, y_train, cv=stratified_folds, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

ROC: 0.88 (+/- 0.03)
Accuracy: 0.93 (+/- 0.01)


Eli5

In [ ]:
!pip install eli5

In [ ]:
import eli5

In [ ]:
index_to_word = {v:k for k,v in count_vect.vocabulary_.items()}

In [ ]:
#index_to_word

In [ ]:
feats = eli5.formatters.as_dataframe.explain_weights_df(lr)
feats_tox = feats[feats['target'] == 1]

top_features_tox = [int(i[1:]) for i in feats_tox.feature if 'BIAS' not in i]


n = 10000
top_tox = []
          
for index in top_features_tox:
  word = index_to_word[index]
  if len(top_tox) < n:
    top_tox.append(word)
  else:
    break



print(top_tox)

['fucking', 'fuck', 'gay', 'bitch', 'suck', 'stupid', 'crap', 'shit', 'damn', 'asshole', 'idiot', 'bullshit', 'dick', 'racist', 'idiotic', 'sucks', 'fat', 'sockpuppet', 'next', 'arab', 'conspiracy', 'moron', 'idiots', 'ignorant', 'penis', 'his', 'life', 'die', 'youtube', 'what the', 'hate', 'fuck you', 'dude', 'ass', 'off', 'in my', 'kill', 'hell', 'blood', 'wtf', 'that they', 'stupidity', 'vagina', 'to hell', 'fu', 'morning', 'fire', 'here is', 'im', 'for his', 'white', 'constructive', 'retarded', 'dumb', 'do something', 'shitty', 'contribution', 'land', 'against me', 'is an', 'heavy', 'eat', 'ur', 'rude', 'participating in', 'participating', 'stop editing', 'you are', 'wiki', 'loser', 'nigger', 'let', 'arrogant', 'he is the', 'jones', 'case and', 'body', 'family', 'girls', 'me to', 'rubbish', 'contribution to', 'fix it', 'disgrace', 'hating', 'ugly', 'breast', 'lewis', 'put the', 'cock', 'ban me', 'get life', 'this guy', 'joke', 'back on', 'but what', 'fascist', 'motherfucker', 'path

In [ ]:
len(top_tox)

10000

Теперь сравним этот список со списками самых частотных слов с нулём в колонке 'toxic', но единичкой в какой-то другой. Данные слова не обязательно указывают на токсичность комментария, а значит их наличие в тренировочной выборке может негативно сказываться на качестве модели. Однако если слово при этом, содержится и в списке severe, то удалять его не будем. 

In [ ]:
delete_lst = []
for word in top_tox:
  if word in severe:
    continue
  elif word in NT_obs or word in NT_thr or word in word in NT_ins or word in NT_ih:
    delete_lst.append(word)
  else:
    continue

In [ ]:
len(delete_lst)

754

Теперь отфильтруем эти слова. 

In [ ]:
def normalize2(text):
  text = re.sub('\n|\t|\s\"', ' ', text)
  text = re.sub(f'[{punctuation}]', ' ', text)
  text = re.sub('\d+', ' ', text)
  text = text.lower()
  #print(type(text))
  for word in delete_lst:
    text = text.replace(word, '')
    #text = re.sub(en_stops, ' ', text)
  text = re.sub('\s+', ' ', text)

  return text

In [ ]:
sample1['comment_text_norm'] = sample1['comment_text'].apply(normalize2)

In [ ]:
sample1.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_norm
128037,acd47a6c06c4d1f8,Fuck Wikipedia!!! \n\nPeace (except to Wikipedia),1,1,1,0,0,0,fuck wikipedia peace cept to wikipedia
208780,c40dc2b2890a95ab,::That is discussion among experts. 1965 war m...,0,0,0,0,0,0,that is discuion among war maybe un ceafire w...
5192,0dd47ea93d47e518,Re-adding anti-barnstar\n Anti-Barnstar,0,0,0,0,0,0,re adding anti barnstar anti barnstar
50057,85c6afcc7c206a1a,"""\nCopying to talk page. serenC """,0,0,0,0,0,0,to talk page renc
109259,484bf6015ebcb2e2,On edit: forgot to actually .,0,0,0,0,0,0,on edit forgot to actually
73349,c43da478b3a8acde,Who is in charge of Russia? Putin or Medevev? ...,0,0,0,0,0,0,who is in charge of ruia putin or medevev i ga...
117583,74384ecc7212b9c8,I had posted this as an edit description but i...,0,0,0,0,0,0,i had posted this as an edit description but i...
21440,3881c4bba57c1b25,"""\n\nNo Tom, I am not saying that you intentio...",0,0,0,0,0,0,no tom i am not saying that you ly added fal ...
151681,810c7aee2b93dd30,"Excuse me, but my Cobain edits are fine and I ...",0,0,0,0,0,0,cu me but my cobain edits are fine and i would...
180172,51e296f21be1829f,"Hi, Welcome. I notice that you're making a num...",0,0,0,0,0,0,hi welcome i notice that you re making a numbe...


Новая бейзлайн регрессия


In [ ]:
X = count_vect.fit_transform(sample1['comment_text_norm'])
y = sample1['toxic']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)

In [ ]:
lr2 = linear_model.LogisticRegression()
lr2.fit(X_train, y_train)

In [ ]:
scores = cross_val_score(lr2, X_train, y_train, cv=stratified_folds, scoring='roc_auc')
print("ROC: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

scores = cross_val_score(lr2, X_train, y_train, cv=stratified_folds, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


ROC: 0.88 (+/- 0.03)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Accuracy: 0.93 (+/- 0.01)


Ничего не изменилось. Наверное, отсеивание мусорных фич нивелировалось тем, что тексты были приведены к ниженему регистру (капс, быть может, является сильным показателем токсичности)

###Стакинг бустингов (как минимум 2 уровня xboost/catboost/lightgbm, можно использовать публичные кернелы любых соревнований) - 3 балла


In [ ]:
!pip install catboost

In [5]:
from catboost import CatBoostClassifier

In [7]:
import xgboost as xgb

In [14]:
from mlxtend.classifier import StackingClassifier

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [ ]:
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from mlxtend.plotting import plot_learning_curves
from mlxtend.plotting import plot_decision_regions use_best_model

In [12]:
from sklearn.ensemble import AdaBoostClassifier

In [10]:
from sklearn.svm import NuSVC, SVC

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier

In [15]:
clf1 = linear_model.LogisticRegression() 
boosting1 = xgb.XGBClassifier(base_estimator=clf1, n_estimators=10, verbosity=0)
boosting2 = CatBoostClassifier(n_estimators=10, verbose=0)
clf3 = RandomForestClassifier(random_state=1)
boosting3 = AdaBoostClassifier(base_estimator=clf3, n_estimators=10)
mc = SVC(probability = True, verbose=False)
sclf = StackingClassifier(classifiers=[boosting1, boosting2, boosting3],
                            use_probas = True, 
                          meta_classifier=mc)

In [ ]:
label = ['XG', 'CAT', 'ADA', 'Stacking Classifier']
clf_list = [boosting1, boosting2, boosting3, sclf]
    
    
for clf, label in zip(clf_list, label):
        
    scores = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
    print ("ROC: %.2f (+/- %.2f) [%s]" %(scores.mean(), scores.std(), label))

    scores = cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy')
    print ("Accuracy: %.2f (+/- %.2f) [%s]" %(scores.mean(), scores.std(), label))
  

ROC: 0.77 (+/- 0.03) [XG]
Accuracy: 0.92 (+/- 0.01) [XG]
ROC: 0.82 (+/- 0.02) [CAT]
Accuracy: 0.93 (+/- 0.00) [CAT]
ROC: 0.87 (+/- 0.02) [ADA]
Accuracy: 0.92 (+/- 0.00) [ADA]
ROC: 0.87 (+/- 0.02) [Stacking Classifier]
Accuracy: 0.92 (+/- 0.00) [Stacking Classifier]


ROC: 0.77 (+/- 0.03) [XG]

Accuracy: 0.92 (+/- 0.01) [XG]

ROC: 0.82 (+/- 0.02) [CAT]

Accuracy: 0.93 (+/- 0.00) [CAT]

ROC: 0.87 (+/- 0.02) [ADA]

Accuracy: 0.92 (+/- 0.00) [ADA]

ROC: 0.87 (+/- 0.02) [Stacking Classifier]

Accuracy: 0.92 (+/- 0.00) [Stacking Classifier]


###Любая нейронная модель (минимум 5 слоев) с Dropout, Pooling и колбеками - 2 балла


In [ ]:
import tensorflow as tf

In [ ]:
def preprocess(text):
    tokens = text.lower().split()
    return tokens

In [ ]:
vocab = Counter()

for text in sample1['comment_text_norm']:
    vocab.update(preprocess(text))

In [ ]:
len(vocab)

31871

In [ ]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 3:
        filtered_vocab.add(word)

In [ ]:
len(filtered_vocab)

7838

In [ ]:
#filtered_vocab

In [ ]:
word2id = {'UNK':1, 'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [ ]:
id2word = {i:word for word, i in word2id.items()}

In [ ]:
X_new = []

for text in sample1['comment_text_norm']:
    tokens = preprocess(text)
    ids = [word2id.get(token, 1) for token in tokens]
    X_new.append(ids)

In [ ]:
MAX_LEN = max(len(x) for x in X_new)

In [ ]:
MAX_LEN 

1354

In [ ]:
X_new = tf.keras.preprocessing.sequence.pad_sequences(X_new, maxlen=MAX_LEN)

In [ ]:
X_train_new, X_valid, y_train_new, y_valid = train_test_split(X_new, y, test_size=0.05, stratify=y)

In [ ]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))

embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=60)(inputs)

conv = tf.keras.layers.Conv1D(kernel_size=5, filters=32, strides=1, 
                                               kernel_regularizer='l2',
                                              activation='relu')(embeddings)

convs1 = []
for ks in [2,3,4,5]:
    conv1 = tf.keras.layers.Conv1D(kernel_size=ks, filters=32, padding='same',strides=1)(conv)
    conv2 = tf.keras.layers.Conv1D(kernel_size=ks, filters=32, padding='same',strides=1, 
                                  kernel_regularizer='l2', activation='relu')(conv1)
    convs1.append(conv2)

concat1 = tf.keras.layers.concatenate(convs1, axis=2)

drop1 = tf.keras.layers.Dropout(0.2)(concat1)

convs2 = []
for ks in [2,3,4,5]:
    conv1 = tf.keras.layers.Conv1D(kernel_size=ks, filters=32, padding='same',strides=1)(drop1)
    conv2 = tf.keras.layers.Conv1D(kernel_size=ks, filters=32, padding='same',strides=1, 
                                  kernel_regularizer='l2', activation='relu')(conv1)
    convs2.append(conv2)

concat2 = tf.keras.layers.concatenate(convs2, axis=2)  

conv3 = tf.keras.layers.Conv1D(kernel_size=5, filters=32, strides=1, 
                                               kernel_regularizer='l2',
                                              activation='relu')(concat2)
pool = tf.keras.layers.MaxPool1D()(conv3)

conv_global = tf.keras.layers.Conv1D(kernel_size=5, filters=32, strides=1)(pool)
flatten = tf.keras.layers.Flatten()(conv_global)
dense = tf.keras.layers.Dense(50, activation='relu')(flatten)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Accuracy(), tf.keras.metrics.AUC()])

In [ ]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 1354)]       0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, 1354, 60)     470400      input_19[0][0]                   
__________________________________________________________________________________________________
conv1d_286 (Conv1D)             (None, 1350, 32)     9632        embedding_18[0][0]               
__________________________________________________________________________________________________
conv1d_287 (Conv1D)             (None, 1350, 32)     2080        conv1d_286[0][0]                 
___________________________________________________________________________________________

In [ ]:
model.fit(X_train_new, y_train_new, 
          validation_data=(X_valid, y_valid),
          batch_size=64,
         epochs=5)

Epoch 1/5
133/133 [==============================] - 382s 3s/step - loss: 1.5677 - accuracy: 0.0000e+00 - auc_7: 0.5712 - val_loss: 0.5604 - val_accuracy: 0.0000e+00 - val_auc_7: 0.6633
Epoch 2/5
133/133 [==============================] - 375s 3s/step - loss: 0.4057 - accuracy: 0.0000e+00 - auc_7: 0.5922 - val_loss: 0.3272 - val_accuracy: 0.0000e+00 - val_auc_7: 0.6608
Epoch 3/5
133/133 [==============================] - 374s 3s/step - loss: 0.3133 - accuracy: 0.0000e+00 - auc_7: 0.6101 - val_loss: 0.3050 - val_accuracy: 0.0000e+00 - val_auc_7: 0.6583
Epoch 4/5
133/133 [==============================] - 357s 3s/step - loss: 0.3054 - accuracy: 0.0000e+00 - auc_7: 0.6216 - val_loss: 0.2985 - val_accuracy: 0.0000e+00 - val_auc_7: 0.6519
Epoch 5/5
133/133 [==============================] - 348s 3s/step - loss: 0.3010 - accuracy: 0.0000e+00 - auc_7: 0.6241 - val_loss: 0.2977 - val_accuracy: 0.0000e+00 - val_auc_7: 0.6555


###Использование эмбедингов в модели (предобученные или обученные с нуля word2vec, fastext, glove) - 1.25 балла

In [ ]:
import gensim

In [ ]:
texts = sample1['comment_text_norm'].tolist()

In [ ]:
w2v = gensim.models.Word2Vec([text.split() for text in texts], size=50, sg=1)

In [ ]:
ft = gensim.models.FastText([text.split() for text in texts], size=60, iter=2)

In [ ]:
weights = np.zeros((len(word2id), 60))

for word, i in word2id.items():
    # вектор паддинга оставим нулевым
    if word == 'PAD':
        continue
    
    # для UNK будем использовать какой-то рандомный вектор
    if word == 'UNK':
        weights[i] = ft['asffaf']
    
    try:
        weights[i] = ft[word]
    
    
    except KeyError:
        # для слов, которых нет в модели тоже возьмем какой-то  рандомный вектор
        weights[i] = ft['agsgsafasdasZF']

In [ ]:
Inputs = tf.keras.layers.Input(shape=(MAX_LEN,))

embeddings = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=60, trainable=False,
                                      weights=[weights])(inputs)

conv = tf.keras.layers.Conv1D(kernel_size=5, filters=32, strides=1, 
                                               kernel_regularizer='l2',
                                              activation='relu')(embeddings)

convs1 = []
for ks in [2,3,4,5]:
    conv1 = tf.keras.layers.Conv1D(kernel_size=ks, filters=32, padding='same',strides=1)(conv)
    conv2 = tf.keras.layers.Conv1D(kernel_size=ks, filters=32, padding='same',strides=1, 
                                  kernel_regularizer='l2', activation='relu')(conv1)
    convs1.append(conv2)

concat1 = tf.keras.layers.concatenate(convs1, axis=2)

drop1 = tf.keras.layers.Dropout(0.2)(concat1)

convs2 = []
for ks in [2,3,4,5]:
    conv1 = tf.keras.layers.Conv1D(kernel_size=ks, filters=32, padding='same',strides=1)(drop1)
    conv2 = tf.keras.layers.Conv1D(kernel_size=ks, filters=32, padding='same',strides=1, 
                                  kernel_regularizer='l2', activation='relu')(conv1)
    convs2.append(conv2)

concat2 = tf.keras.layers.concatenate(convs2, axis=2)  

conv3 = tf.keras.layers.Conv1D(kernel_size=5, filters=32, strides=1, 
                                               kernel_regularizer='l2',
                                              activation='relu')(concat2)
pool = tf.keras.layers.MaxPool1D()(conv3)

conv_global = tf.keras.layers.Conv1D(kernel_size=5, filters=32, strides=1)(pool)
flatten = tf.keras.layers.Flatten()(conv_global)
dense = tf.keras.layers.Dense(50, activation='relu')(flatten)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

model2 = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model2.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Accuracy(), tf.keras.metrics.AUC()])

In [ ]:
model2.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 1354)]       0                                            
__________________________________________________________________________________________________
embedding_17 (Embedding)        (None, 1354, 60)     470400      input_16[0][0]                   
__________________________________________________________________________________________________
conv1d_267 (Conv1D)             (None, 1350, 32)     9632        embedding_17[0][0]               
__________________________________________________________________________________________________
conv1d_268 (Conv1D)             (None, 1350, 32)     2080        conv1d_267[0][0]                 
___________________________________________________________________________________________

In [ ]:
model2.fit(X_train_new, y_train_new, 
          validation_data=(X_valid, y_valid),
          batch_size=64,
         epochs=5)

Epoch 1/5
133/133 [==============================] - 341s 3s/step - loss: 1.5778 - accuracy: 0.0000e+00 - auc_6: 0.5920 - val_loss: 0.5733 - val_accuracy: 0.0000e+00 - val_auc_6: 0.6665
Epoch 2/5
133/133 [==============================] - 341s 3s/step - loss: 0.3981 - accuracy: 0.0000e+00 - auc_6: 0.6123 - val_loss: 0.3222 - val_accuracy: 0.0000e+00 - val_auc_6: 0.6634
Epoch 3/5
133/133 [==============================] - 342s 3s/step - loss: 0.3148 - accuracy: 0.0000e+00 - auc_6: 0.6053 - val_loss: 0.3056 - val_accuracy: 0.0000e+00 - val_auc_6: 0.6581
Epoch 4/5
133/133 [==============================] - 342s 3s/step - loss: 0.3040 - accuracy: 0.0000e+00 - auc_6: 0.6179 - val_loss: 0.3010 - val_accuracy: 0.0000e+00 - val_auc_6: 0.6526
Epoch 5/5
133/133 [==============================] - 345s 3s/step - loss: 0.3044 - accuracy: 0.0000e+00 - auc_6: 0.6055 - val_loss: 0.3206 - val_accuracy: 0.0000e+00 - val_auc_6: 0.6516
